# CMSE 401 - Final Exam
This is an open Internet exam.  Feel free to use anything on the Internet with one important exception...

- **DO NOT** communicate live with other people during the exam (either verbally or on-line).  The goal here is to find answers to problems as you would in the real world.  

You will be given **60 minutes (wishful thinking by the instructor, you will be given the entire exam time if needed)** to complete this Exam.  Use your time wisely. 

**HINTS:**
- Neatness and grammar is important.  We will ignore all notes or code we can not read or understand.
- Read the entire exam from beginning to end before starting.  Not all questions are equal in points vs. time so plan your time accordingly.   
- Some of the information provided my be a distraction. Do not assume you need to understand everything written to answer the questions. 
- Spaces for answers are provided. Delete the prompting text such as "Put your answer to the above question here" and replace it with your answer. Do not leave the prompting text with your answer.
- Do not assume that the answer must be in the same format of the cell provided. Feel free to change the cell formatting (ex. markdown to code) or add additional cells as needed to provide your answer.
- When a questions asks for an answer "**in your own words**" it is still okay to search the Internet for the answer as a reminder, however, we would like you to do more than cut and paste.  Make the answer your own. 
- If you get stuck, try not to leave an answer blank. It is better to include some notes or stub functions so we have an idea about your thinking process so we can give you partial credit.   
- Always provided links to any references you find helpful. 
- Feel free to delete the provided check marks (&#9989;) as a way to keep track of which questions you have successfully completed. 



### Exam Summary
1. (25 points) [Makefiles](#Q1)
2. (25 points) [Distributed Memory](#Q2)
2. (25 points) [Shared Memory](#Q3)
3. (25 points) [Accelerators](#Q4)

---
<a name="Q1"></a>
# Question 1 - (25 points) Makefiles

As we mentioned in class. Makefiles are fairly simple in concept but are an entire programming language so can get very advanced. At its simplest level a makefile consists of rules in the following format:

```
target : prerequisite_file1 prerequisite_file2 prequiste_file3 ...
   #Bash build command(s) to make the target file from the prerequisite files
```

Makefiles can be thought of as Directed Acyclic Graphs (DAGs).  For example the makefile found in this [file](https://colbrydi.github.io/images//Makefile) can be visualized as the following dependency graph. 

<img src="https://colbrydi.github.io/images/make.png" width=50%>

Notice that files that need to be "made" on the same level could be built at the same time. This means we could build them in parallel. For example, in the graph shown above, if we assume that their prerequisite file already exist, ```make2dot.o``` ```y.tab.o``` and ```lex.y.cc``` could can be made at the same time.  While the ```make2dot``` executable can not be made until ```lex.y.o``` is made. 

The ```make``` program already has a ```-j``` (jobs) option which can be used to leverage the DAG and run in parallel. For example, the following command would run make with four (4) separate tasks running in parallel:

    make -j4 
    
    

&#9989; <font color=red>**Question 1.a**</font>: Does make use shared memory, shared network, accelerator or a hybrid method for it's parallelization.  How did you come to this answer (if this is a guess, explain your logic)?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 1.b**</font>: The ```make``` program reports an error if the generated graph produces a dependency cycle.  This means that the arrows in the graph are never allowed to form a loop. This guarantee of an acyclic graph ensures that running ```make``` in parallel will _**not**_ cause a common parallel error when using the ```-j``` option.  What is that common error and why does the acyclic graph prevent it?

<font color=red>Put the answer to the above question here</font>

The makefile formate is more than the simple rules shown above. For example, the following makefile uses variables, a concept called "wildcards" and the [ImageMagic](https://imagemagick.org)  ```convert``` command to take all of the ```png``` files in the current directory and convert them to compressed ```jpg``` files in a folder named images.  

```bash

# Creates a list of png images in currect directory
INPUT_IMAGES=$(wildcard *.png)

# Define the output directory
DIR=./images

# Define the output image name
OUTPUT_IMAGES=$(INPUT_IMAGES:%.png=%.jpg)

all: images $(OUTPUT_IMAGES)

# Convert a single image
%.jpg : %.png
	convert $< $(DIR)/$@

# Create the output directory
images : 
	mkdir -p images

# Delete the output directory
clean :
	rm -rf images

```

This makefile creates a very simple DAG where all of the images are on the same level and can be run in parallel. In other words, converting one image to ```jpg``` has nothing to do with converting another image to ```jpg``` because they only have one dependency (i.e. the associated```png``` file) which they do not share. 


&#9989; <font color=red>**Question 1.c:**</font>:  The above makefile is essentially a pleasantly parallel problem.  If it takes 0.3 seconds to convert a single image, what is a "best case" estimation on how long it would take to convert 10,000 images on a 40 core machine using the```make -j40``` command. Make sure you explain your answer by showing your work?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 1.d**</font>: Now assume that approximately 60% of the time spent converting an image is taken up by File I/O (input and output).  Also assume that due to the serialization of the disk drive, the computer can only read one file at a time (i.e. you can't parallelize the File I/O no matter how many threads you give the problem).  Now, what is a "best case" estimation on how long it would take to convert 10,000 images (make sure you explain your answer and/or show your work with units)?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 1.e**</font>: One common user error when generating a makefile is to forget to include some of the dependencies (prerequisite) required for the build command.   This is a common problem because, ```make``` will not return an error unless a listed dependency doesn't exist or one of the build commands fail. When running in serial, forgetting a dependency in the rule is often not a big deal and the ```make``` command still manages to build the files in the right order.  However, when running in parallel forgetting a dependency may cause what type of problem?  Explain your answer and/or give an example.

<font color=red>Put the answer to the above question here</font>

----
<a name="Q2"></a>
# Question 2 - (25 points) Distributed Memory Jobs

You have been asked to design an agent-based trading model similar to the one we built in [HW6](./HW6_Solution/rumor_mill.c).  Each "agent" is a cell on a grid that can only "see" it's four neighboring cells (Up, Down, Left Right).  The Principal Investigator (PI or lead researcher) on your project would like to scale this code to fairly large systems and has asked that you split up the world into a grid of of $N$ cells (not slices as we did in the homework solution).  For example, a grid with $N=16$ tasks with four (4) rows and four (4) columns would have the following layout (task ranks are on top (**<font color=blue>blue</font>**), grid coordinates are in parentheses (**black**)):

<img src="https://computing.llnl.gov/tutorials/mpi/images/Cartesian_topology.gif">


<font color=red>**Question 2.a:**</font> Given an MPI task's Rank (variable name $rank$), total number of rows (variable name $nrows$), and total number of columns (variable name $ncols$). Write a short piece of C code that would calculate the grid coordinates ($row$ and $col$) of a task (i.e. the calculate the number in the parentheses given the blue number for an arbitrary size grid). For this question you can assume that the total number of tasks ($N$) equals $nrows \times ncols$.

<font color=red>Put the answer to the above question here</font>

<font color=red>**Question 2.b:**</font> Now, given all of the variables we have so far ($rank$, $ncols$, $nrows$, $col$ and $row$). We want to pre-calculate the ranks of a task's neighboring cells. Call these ranks $up$, $down$, $left$ and $right$.  For this example, assume that the boarders do not wrap and that a cell on the edge has "no neighbor" which we will indicate with a rank of -1. The following code calculates the $up$ and $down$ variables. Please finish the code to include $left$ and $right$

<font color=red>Put the answer to the above question in the code cell below</font>

Like the Homework solution, each task in the grid maintains a state of the simulation as an array of integers and needs to transmit the state of the array's edges to its neighbors.  The following code (modified from HW6) transmits the edges:

```c++
    // Send up 
    if (up == -1) {
	   MPI_Recv(my_world[which][sz_y+1],sz_x+2,MPI_BYTE,down,up_tag,MPI_COMM_WORLD,&status);
	} else {
	   MPI_Send(my_world[which][1],sz_x+2,MPI_BYTE,up,up_tag,MPI_COMM_WORLD);
	   MPI_Recv(my_world[which][sz_y+1],sz_x+2,MPI_BYTE,down,up_tag,MPI_COMM_WORLD,&status);
	}

	// Send down
	if (down == -1) {
	   MPI_Send(my_world[which][sz_y],sz_x+2,MPI_BYTE,down,down_tag,MPI_COMM_WORLD);
	} else {
	   MPI_Send(my_world[which][sz_y],sz_x+2,MPI_BYTE,down,down_tag,MPI_COMM_WORLD);
	   MPI_Recv(my_world[which][0],sz_x+2,MPI_BYTE,up,down_tag,MPI_COMM_WORLD,&status);
	}

    // Copy my left vertical vector border #QUESTION 2d
    for(int r=1; r < sz_y; r++)
       vertical_vec[r] = my_world[which][r][1];

    // Send left
    if (left != -1)
        MPI_Recv(vertical_vec,sz_y,MPI_BYTE,left,left_tag,MPI_COMM_WORLD,&status);
	} else {
	   MPI_Send(vertical_vec,sz_y,MPI_BYTE,left,left_tag,MPI_COMM_WORLD);
	   MPI_Recv(vertical_vec,sz_y,MPI_BYTE,left,left_tag,MPI_COMM_WORLD,&status);
	}
    
    // Paste left vector to my right boarder #QUESTION 2d
    for(int r=1; r < sz_y; r++)
       my_world[which][r][sz_x+2] = vertical_vec[r];
    
    // Copy my right vertical vector boarder  #QUESTION 2d
    for(int r=1; r < sz_y; r++)
       vertical_vec[r] = my_world[which][r][sz_y+1];

    // Send Right
    if (left != -1)
        MPI_Recv(vertical_vec,sz_y,MPI_BYTE,right,right_tag,MPI_COMM_WORLD,&status);
	} else {
	   MPI_Send(vertical_vec,sz_y,MPI_BYTE,right,right_tag,MPI_COMM_WORLD);
	   MPI_Recv(vertical_vec,sz_y,MPI_BYTE,right,right_tag,MPI_COMM_WORLD,&status);
	}
    
    // Paste right vector to my left boarder #QUESTION 2d
    for(int r=1; r < sz_y; r++)
       my_world[which][r][0] = vertical_vec[r];
        
```

&#9989; <font color=red>**Question 2.c**</font>: Assuming the above code works in the context of the entire program.  Explain why the code for transmitting the $left$ and $right$ vertical edges has the extra copy/paste loops (indicated by the ```#QUESTION 2d``` tags in comments above) that we do not need in the up and down.

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 2.d**</font>: It is probably true that using non-blocking send and receives would be slightly faster.  Explain why you might **NOT** want to use non-blocking messages?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 1.e:**</font>: Assume you know that some parts of the task grid has more work to do than others. The fastest time the simulation can run will depend on the slowest component.  For example, the corners (ex: ranks 0, 3, 12 and 15 in the above figure) take only 30 seconds to run per time step; the edges (ex: 1,2,4,7,8,11,13 and 14 in the above figure) take 2 minutes to run per time step and the middle parts (ex: 5, 6, 9 and 10 in the above figure) take 4 minutes to run per time step.  In this case all 16 tasks would take $4 \text{minutes} \times \text{number of time steps}$ to run and most of the tasks would be sitting around doing nothing waiting for the middle tasks.  Describe what you could do to make the entire runtime of the simulation faster? 

<font color=red>Put the answer to the above question here</font>

---
<a name="Q3"></a>

# Question 3 - (25 points) Shared Memory Jobs


A simple matrix multiply between matrix $A$ (with $i$ rows and $j$ columns) and matrix $B$ (with $j$ rows and $k$ columns) into matrix $C$ (with $i$ rows and $k$ columns) is defined as follows:

$$A_{(i\times j)} \times B_{(j\times k)} = C_{(i\times k)} $$

Where each $i,k$ element in $C$ (aka $c_{i,j}$) is calculated using the dot product of the $i^{th}$ row of $A$ by the $j^{th}$ column of $B$:

$$c_{i,j} = a_{i,0}b_{0,j} + a_{i,1}b_{1,j} + \dots + a_{i,k}b_{k,j}$$

The following figure is an example depiction of this algorithm:

<img src="https://lh3.googleusercontent.com/bV-ObD9gwRD4PIcGDEzCP51yGk-f8sx5BUO4JXj9u-KUwYpUJbqRv7A46cwNChKW3S2WD6n3Cg=w500" width=40%>


More information about matrix multiply can be found [here](https://en.wikipedia.org/wiki/Matrix_multiplication).

The following C code snip-it can be used to calculate a matrix multiplication between $A$ and $B$:

```c
//Simple Matrix Multiply
for(int i=0;i<sz_i;i++){
    for(int k=0;k<sz_k;k++){
        C[i][k] = 0;
        for(int j=0;k<sz_j;j++){
            C[i][k] += A[i][j] * B[j][k];
        }
    }
}
```

&#9989; <font color=red>**Question 3.a**</font>: Write an OpenMP pragma for loop to to efficiently parallelize the above matrix?  Where would this command go in the above example?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 3.b**</font>:  Which OpenMP scheduling option would best be suited for this problem and why?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 3.c**</font>:  Why would you **NOT** want to use OpenMP if the $A$ and $B$ matrices are really small? 

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 3.d** </font>: Matrix multiply can also be implemented efficiently using GPUs and MPI.  In your own words, explain when each is the best choice for solving a problem and why.

* <font color=red>
OpenMP - Put your answer here explaining when OpenMP is the best choice for solving matrix multiply and why.
</font>

* <font color=red>CUDA - Put your answer here explaining when CUDA is the best choice for solving matrix multiply and why.</font>

* <font color=red>MPI - Put your answer here explaining when MPI is the best choice for solving matrix multiply and why.</font>

&#9989; <font color=red>**Question 3.e**</font>:  There are many c/c++ compatible Linear Algebra (LA) program Libraries available that can do matrix multiply.  Find a linear algebra library already installed on the MSU HPCC and provide a link to some documentation for that library.

<font color=red>Put the answer to the above question here</font>

----
<a name="Q4"></a>

# Question 4 - (25 points)  CUDA Texture Memory


> Like constant memory, texture memory is cached on a GPU's chip, so in some situations it will provide higher effective bandwidth by reducing memory requests to off-chip DRAM. Specifically, texture caches are designed for graphics applications where memory access patterns exhibit a great deal of spatial locality. In a computing application, this roughly implies that a thread is likely to read from an address “near” the address that nearby threads read, as shown in Figure

<img src="http://1.bp.blogspot.com/-Y-Kga_6J0WU/UQ38DOMpYdI/AAAAAAAAADQ/ejIIjQmKiM8/s640/1.png">

> Note:
Arithmetically, the four addresses shown are not consecutive, so they would not be cached together in a typical CPU caching scheme. But since GPU texture caches are designed to accelerate access patterns such as this one, you will see an increase in performance in this case when using texture memory instead of global memory.


Consider the following example CUDA code found on stack overflow [REF](https://stackoverflow.com/questions/14334251/cuda-image-average-filter)

```c++
texture<unsigned char, cudaTextureType2D> tex8u;

//Box Filter Kernel For Gray scale image with 8bit depth
__global__ void box_filter_kernel_8u_c1(unsigned char* output,const int width, const int height, const size_t pitch, const int fWidth, const int fHeight)
{
    int xIndex = blockIdx.x * blockDim.x + threadIdx.x;
    int yIndex = blockIdx.y * blockDim.y + threadIdx.y;

    const int filter_offset_x = fWidth/2;
    const int filter_offset_y = fHeight/2;

    float output_value = 0.0f;

    //Make sure the current thread is inside the image bounds
    if(xIndex<width && yIndex<height)
    {
        //Sum the window pixels
        for(int i= -filter_offset_x; i<=filter_offset_x; i++)
        {
            for(int j=-filter_offset_y; j<=filter_offset_y; j++)
            {
                //No need to worry about Out-Of-Range access. tex2D automatically handles it.
                output_value += tex2D(tex8u,xIndex + i,yIndex + j);
            }
        }

        //Average the output value
        output_value /= (fWidth * fHeight);

        //Write the averaged value to the output.
        //Transform 2D index to 1D index, because image is actually in linear memory
        int index = yIndex * pitch + xIndex;

        output[index] = static_cast<unsigned char>(output_value);
    }
}


void box_filter_8u_c1(unsigned char* CPUinput, unsigned char* CPUoutput, const int width, const int height, const int widthStep, const int filterWidth, const int filterHeight)
{

    /*
     * 2D memory is allocated as strided linear memory on GPU.
     * The terminologies "Pitch", "WidthStep", and "Stride" are exactly the same thing.
     * It is the size of a row in bytes.
     * It is not necessary that width = widthStep.
     * Total bytes occupied by the image = widthStep x height.
     */

    //Declare GPU pointer
    unsigned char *GPU_input, *GPU_output;

    //Allocate 2D memory on GPU. Also known as Pitch Linear Memory
    size_t gpu_image_pitch = 0;
    cudaMallocPitch<unsigned char>(&GPU_input,&gpu_image_pitch,width,height);
    cudaMallocPitch<unsigned char>(&GPU_output,&gpu_image_pitch,width,height);

    //Copy data from host to device.
    cudaMemcpy2D(GPU_input,gpu_image_pitch,CPUinput,widthStep,width,height,cudaMemcpyHostToDevice);

    //Bind the image to the texture. Now the kernel will read the input image through the texture cache.
    //Use tex2D function to read the image
    cudaBindTexture2D(NULL,tex8u,GPU_input,width,height,gpu_image_pitch);

    /*
     * Set the behavior of tex2D for out-of-range image reads.
     * cudaAddressModeBorder = Read Zero
     * cudaAddressModeClamp  = Read the nearest border pixel
     * We can skip this step. The default mode is Clamp.
     */
    tex8u.addressMode[0] = tex8u.addressMode[1] = cudaAddressModeBorder;

    /*
     * Specify a block size. 256 threads per block are sufficient.
     * It can be increased, but keep in mind the limitations of the GPU.
     * Older GPUs allow maximum 512 threads per block.
     * Current GPUs allow maximum 1024 threads per block
     */

    dim3 block_size(16,16);

    /*
     * Specify the grid size for the GPU.
     * Make it generalized, so that the size of grid changes according to the input image size
     */

    dim3 grid_size;
    grid_size.x = (width + block_size.x - 1)/block_size.x;  /*< Greater than or equal to image width */
    grid_size.y = (height + block_size.y - 1)/block_size.y; /*< Greater than or equal to image height */

    //Launch the kernel
    box_filter_kernel_8u_c1<<<grid_size,block_size>>>(GPU_output,width,height,gpu_image_pitch,filterWidth,filterHeight);

    //Copy the results back to CPU
    cudaMemcpy2D(CPUoutput,widthStep,GPU_output,gpu_image_pitch,width,height,cudaMemcpyDeviceToHost);

    //Release the texture
    cudaUnbindTexture(tex8u);

    //Free GPU memory
    cudaFree(GPU_input);
    cudaFree(GPU_output);
}
```

The steps for using texture memory are as follows:

1. Declare the texture memory in CUDA.
1. Bind the texture memory to your texture reference in CUDA.
1. Read the texture memory from your texture reference in CUDA Kernel.
1. Unbind the the texture memory from your texture reference in CUDA

&#9989; <font color=red>**Question 4.a:**</font> What CUDA Commands are used in the above example to bind and unbind the texture memory?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 4.b:**</font> If texture memory is faster, why don't we use it for all CUDA memory access?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 4.c:**</font> In your own words, describe why is the ```tex2D``` neeed and what is it doing?

<font color=red>Put the answer to the above question here</font>

Running a device query command in the course BCCD directory results in the following output on dev-intel18:

```
Device 0:
	name = Tesla K80
	CUDA capability major.minor version = 3.7
	multiProcessorCount = 13
	totalGlobalMem = 12799574016 bytes
	sharedMemPerBlock = 49152 bytes
	regsPerBlock = 65536
	warpSize = 32
	memPitch = 2147483647 bytes
	maxThreadsPerBlock = 1024
	maxThreadsDim = 1024 x 1024 x 64
	maxGridSize = 2147483647 x 65535 x 65535

	memPitch = 2147483647 bytes
	textureAlignment = 512 bytes
	clockRate = 0.82 GHz
	deviceOverlap = Yes
	kernelExecTimeoutEnabled = No
	integrated = No
	canMapHostMemory = Yes
	computeMode = Default (multiple host threads can use this device simultaneously)
	concurrentKernels = Yes
	ECCEnabled = No
	tccDriver = No
```

&#9989; <font color=red>**Question 4.d:**</font> How would you change the above code to use the maximum block size on this card?

<font color=red>Put the answer to the above question here</font>

&#9989; <font color=red>**Question 4.e:**</font> The K80 nodes have 4 K80 cards and each card is equivalent to two k40 cards with the specification shown in Device 0 above.  

Assume that your ```main``` program is written to process multiple images at a time.  Also assume the code can is running 8 cores in parallel using openMP and each core is able to talk to a different K40 device (like the one listed above).  Rewrite the following submission script to request 8 cores and 8 GPUs instead of just one.

<font color=red>Modify the following code cell</font>

---------
### Congratulations, you're done with your EXAM

Now, you just need to submit this assignment by uploading it to the course <a href="https://d2l.msu.edu/">Desire2Learn</a> web page for today's dropbox.

&#9989; <font color=red>**DO THIS:**</font>
- Download the Notebook to your desktop with the filename using the format **"<NETID\>_Midterm-Exam.ipynb"**.  Replace <NETID\> in the filename with your personal MSU NetID (the stuff that comes before the @ symbol in your msu email address).
- Upload the newly renamed notebook to the D2L dropbox. 


&#169; Copyright 2019,  Michigan State University Board of Trustees